O teu grupo foi encarregado de fazer uma **interface para cadastro e monitoramento** de restaurantes no iFood. Desenvolvam um sistema com **menu** que tenha as seguintes características:
1. Cadastro de um novo restaurante, da forma \[`nome_restaurante`, \[`lat`,`lon`\], `n_pedidos`\]
1. Criar cardápio.
1. Registrar histórico de vendas.
1. Verificar restaurantes mais próximos de uma localidade.
1. Mostrar restaurantes
1. Mostrar cardápio do restaurante
1. Realizar venda


Você terá em mãos algumas funções que implementam a manipulação de arquivos, para viabilizar o projeto.


## Membros do Grupo 3
* Thiago de Oliveira Albuquerque
* Paloma
* José Diniz
* Donizetti Junior

In [27]:
# print('feijão'.encode('utf-8').decode('utf-8'))

feijão


> ## Execuções preliminares de pré-requisitos para iniciar o sistema

In [28]:
# from google.colab import files
# uploaded = files.upload()

In [29]:
!pip install haversine

> ### Por favor, alterar o diretório se necessário para execução do projeto.
 Não conseguimos encontrar uma solução para essa questão. Foi tentado definir as funções do arquivo `manipulate_data` diretamente no _notebook_, contudo, um erro envolvendo o parâmetro `__file__` ocorreu de modo que inviabilizou a solução proposta.

In [30]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# change the current directory
# to specified directory

shared_folder_name = 'Colab Notebooks/Vem Ser Tech ADA/Logica de Programacao I/Projeto Modulo 1/'
shared_folder_path = f'/content/drive/My Drive/{shared_folder_name}'

if shared_folder_path not in sys.path:
    sys.path.append(shared_folder_path)
os.chdir(shared_folder_path)
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Colab Notebooks/Vem Ser Tech ADA/Logica de Programacao I/Projeto Modulo 1'

In [31]:
from manipulate_data import load_json_file, save_json_file

In [32]:
# script = '''
# # if [-d ./json]; then
# #   mkdir ./json
# # fi
# mkdir -p ./json
# '''
# with open('./script.sh', 'w') as file:
#   file.write(script)

# !bash ./script.sh

> ## Função auxiliar responsável por buscar um restaurante pelo nome

In [33]:
from typing import List

def buscar_restaurante(rest_nome: str, restaurantes: List) -> List:
  restaurante = []
  for i in range(len(restaurantes)):
    if rest_nome == restaurantes[i][0]:
      # Selecionando o restaurante i; 0 é o primeiro, 1 é o segundo, etc
      restaurante = restaurantes[i]

  if not restaurante: # Listas vazias são consideradas False em Python; not False == True
    print("Restaurante não encontrado.")
    return []
  else:
    return restaurante

> ## Função auxiliar responsável pela criação do cardápio

In [34]:
from typing import List
# 1- Criar cardápio (Função interna à cadastrar_restaurante)
def criar_cardapio(pasta = './cadastro_json') -> List:
  print('Cadastre no máximo 20 itens no cardápio. Finalize pressionando \'0 (zero)\'.')
  # novo_cardapio = [list(item for item in input(f'Item {i}: ') if item not in '0') for i in range(1, 21)] # Como fazer parar quando digitar 0?
  cardapio = dict()
  print('Insira o preço para cada item.')
  novo_cardapio = []
  for i in range(1, 21):
    item = input(f'Item {i}: ').lower()
    if item in '0':
      break
    else:
      preco = input(f'Preço do item {item} (Use o ponto como separador): ')

    cardapio.update({item:preco})
  # Fim do laço for
  novo_cardapio.append(cardapio)
  return novo_cardapio
# Fim da função criar_cardapio

> ## Função auxiliar para realizar vendas referentes a um único item

In [35]:
# Função realizar_venda
def realizar_venda(pedidos: int, vendas: float, preco: float) -> tuple[List, List]:
  qtd_vendas = int(input("Digite a quantidade vendida: "))
  venda_total = qtd_vendas * preco
  # Atualizando o número de pedidos
  pedidos = str(pedidos + qtd_vendas)
  # Atualizando o valor em vendas
  vendas = str(vendas + venda_total)
  return pedidos, vendas
# Fim da função realizar_venda

> ## 1 -Cadastrar novo restaurante

In [36]:
# 1 - Cadastrar novo restaurante
def cadastrar_restaurante(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> None:
  print('Por favor, indique as seguintes informações referentes ao restaurante.')
  restaurantes_salvos = load_json_file(pasta, arquivo)

  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos
  novo_restaurante = list((
      # Restaurante:
      input('Nome do restaurante: ').lower(), # 0
      # Endereço:
      {
        'Latitude': input('Latitude (Use o ponto como separador): '), # 1.0
        'Longitude': input('Longitude (Use o ponto como separador): ') # 1.1
      }, # 1
      # Único telefone para contato
      input('Telefone para contato: '), # 2
      # Número de pedidos na fase de cadastro:
      '0', # 3
      # Valor total em vendas
      '0', # 4
      # Cardápio (obrigatório criar restaurante com cardápio)
      criar_cardapio(pasta) # 5
    )
  )
  # Fim da lista que armazena informações do novo restaurante

  #              rests[0]     rests[1]      rests[2]         rests[3]               rest[4]            rests[5]
  colunas = ['Restaurante', 'Localizacao', 'Contato', 'Pedidos Realizados', 'Valor Total em Vendas', 'Cardapio'] # 0 1 2 3 4 5

  # Adicionando o restaurante cadastrado à lista de restaurantes previamente cadastrados
  restaurantes.append(novo_restaurante)
  save_json_file(
        folder_name = pasta,
        file_name   = 'restaurantes',
        data        = restaurantes,
        column_name = colunas
  )
# Fim da função cadastrar_restaurante

> ## 2 - Registrar histórico de vendas

In [37]:
# 2 - Registrar histórico de vendas
def registrar_hist_vendas(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> None:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None

  rest_nome = input("Digite o nome do restaurante: ").lower()
  restaurante = buscar_restaurante(rest_nome, restaurantes)

  if not restaurante: # Listas vazias são consideradas False em Python
    return None
  # print(*restaurante)

  rest_nome, localizacao, contato = restaurante[0], restaurante[1], restaurante[2]
  pedidos, vendas, cardapio = int(restaurante[3]), float(restaurante[4]), restaurante[5][0]

  item = '1'
  while item not in '0':
    item = input("Digite o nome do item vendido (digite 0 para encerrar): ").lower()
    if item not in cardapio.keys() and item not in '0':
      print("Item não encontrado no cardápio.")
    elif item in '0':
      continue
    else:
      item_cardapio = item
      preco = float(cardapio.get(item_cardapio))
      # Atualizado o número de pedidos e o valor em vendas
      restaurante[3], restaurante[4] = realizar_venda(pedidos, vendas, preco)
  #Fim do while

  #              rests[0]     rests[1]      rests[2]         rests[3]               rest[4]            rests[5]
  colunas = ['Restaurante', 'Localizacao', 'Contato', 'Pedidos Realizados', 'Valor Total em Vendas', 'Cardapio'] # 0 1 2 3 4 5
  save_json_file(pasta, 'restaurantes', restaurantes, colunas)
  # Fim do if-else
# Fim da função registrar_hist_vendas

> ## 3 - Verificar restaurantes mais próximos de uma localidade dentro de um raio específico

In [38]:
from typing import List
from haversine import haversine, Unit # Unit pode ser utilizado para alterar a unidade de medida da grandeza 'distância'

# 3 - Verificar restaurantes mais próximos de uma localidade dentro de um raio
def verificar_rest_proximos(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> List:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None

  restaurantes_proximos = []

  rest_nome = input("Digite o nome do restaurante de referência: ").lower()
  restaurante_ref = buscar_restaurante(rest_nome, restaurantes)
  if not restaurante_ref: # Listas vazias são consideradas False em Python
    return None

  raio = int(input('Indique o raio de busca (inteiro em km): '))
  # latitude de referência
  latitude_ref = float(restaurante_ref[1].get('Latitude'))
  # longitude de referência
  longitude_ref = float(restaurante_ref[1].get('Longitude'))
  # localizacao de referência
  localizacao_ref = (latitude_ref, longitude_ref)
  for restaurante in restaurantes:
    # Verificação necessária para não considerar o próprio restaurante de referência como próximo a si mesmo
    if restaurante[0] != rest_nome:
      latitude = float(restaurante[1].get('Latitude'))
      longitude = float(restaurante[1].get('Longitude'))
      localizacao = (latitude, longitude)
      # Por padrão, a função haversine retorna a distância entre dois pontos da Terra em quilômetros
      distancia_km = haversine(localizacao_ref, localizacao)
      if distancia_km <= raio:
        rest_proximo = restaurante[0]
        restaurantes_proximos.append(rest_proximo)
        pad = 30
        # pad_nome = len(rest_proximo) + 10
        distancia_km_str = f'{distancia_km:.2f}' + 'km'
        # pad_distancia = len(distancia_km_str) + 10
        print(
        f'''
        {rest_proximo:<{pad}}<{5 * '='}>{distancia_km_str:^{pad}}
        ''')
      # Fim do if interno
    # Fim do if externos
  # Fim do laço for
  return restaurantes_proximos
# Fim da função verificar_rest_proximos
# CONVERTER LATITUDE E LONGITUDE PARA QUILÔMETROS;
# CONSIDERAR PONTOS CARDEAIS

> ## 4 - Mostrar restaurantes

Definição da função utilizada para mostrar as informações de todos os restaurantes em um formato amigável e legível.

In [39]:
# 4 - Mostrar restaurantes
def mostrar_restaurantes(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> None:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None

  iguais = '=' * 70
  for restaurante in restaurantes:
    valor_vendas = 'R$' + f'{float(restaurante[4]):.2f}'
    pad = 30
    print(
    f'''{iguais}
    {'Restaurante':<{pad}}:{restaurante[0]:^{pad}}
    {'Localização':<{pad}}
      {'Latitude':<{pad-2}}:{restaurante[1].get('Latitude'):^{pad}}
      {'Longitude':<{pad-2}}:{restaurante[1].get('Longitude'):^{pad}}
    {'Contato':<{pad}}:{restaurante[2]:^{pad}}
    {'Qtd de Pedidos Realizados':<{pad}}:{restaurante[3]:^{pad}}
    {'Valor Total em Vendas':<{pad}}:{valor_vendas:^{pad}}
    {'Cardápio':<{pad}} '''
    )
    cardapio = restaurante[5][0]
    for item in cardapio:
      pad_item = pad # + len(item)
      preco = 'R$ ' + cardapio.get(item)
      pad_preco = pad # len(preco)
      print(f'      {item:<{pad_item-2}}:{preco:^{pad_preco}}')
    # Fim do laço for interno
  # Fim do laço for externo
  print(iguais)
# Fim da função mostrar_restaurantes

> ## 5 - Mostrar cardápio

In [40]:
# 5 - Mostrar cardápio
def mostrar_cardapio(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> None:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None

  rest_nome = input('Digite o nome do restaurante do qual deseja consultar o cardápio: ').lower()
  restaurante = buscar_restaurante(rest_nome, restaurantes)
  #  while not restaurante:
  if not restaurante:
    print('Restaurante não encontrado.')
    return None

  iguais = '=' * 50
  pad = 0 # 25
  print(
  f'''{iguais}
  {'Cardápio':<{pad}}:'''
  )
  cardapio = restaurante[5][0]
  for item in cardapio:
    pad_item = 25 # + len(item)
    preco = 'R$ ' + cardapio.get(item)
    pad_preco = 25 # len(preco)
    print(f'    {item:<{pad_item-2}}:{preco:^{pad_preco-2}}')
  # Fim do for
  # Imprimir só o nome dos restaurantes
  print(iguais)
# Fim da função mostrar_cardapio

> ## 6 - Excluir um restaurante

In [41]:
# 6 - Excluir um restaurante
def excluir_restaurante(pasta: str = './cadastro_json', arquivo: str = 'restaurantes') -> None:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None


# Fim da função excluir_restaurante

> ## 7 - Excluir todos os restaurantes

In [42]:
import os

# 7 - Excluir todos os restaurantes
def excluir_tudo(pasta: str = './cadastro_json', arquivo: str = 'restaurantes.json') -> None:
  restaurantes_salvos = load_json_file(pasta, arquivo)
  restaurantes = [] if restaurantes_salvos == None else restaurantes_salvos

  if not restaurantes:
    print('Não há restaurantes cadastrados.')
    return None
  confirmacao = input('Tem certeza disso? Digite \'0\' para não excluir!')
  if confirmacao not in '0':
    restaurantes.clear()
    os.remove(pasta + '/' + arquivo)
    print('Todos os dados foram removidos!')
  else:
    print('Os dados não foram removidos!')
# Fim da função excluir_tudo

# Programa principal.

In [43]:
from IPython.display import clear_output

if __name__ == '__main__':
  pasta = './cadastro_json'
  arquivo = 'restaurantes'

  sair = False
  while not sair:
    # Executa o comando para limpar a saída
    clear_output(wait = True)

    print(
      '''
      1 - Cadastrar novo restaurante
      2 - Registrar histórico de vendas
      3 - Verificar restaurantes mais próximos de uma localidade (lat, long)
      4 - Mostrar restaurantes
      5 - Mostrar cardápio de um restaurante
      6 - Excluir um restaurante (Não implementado)
      7 - Excluir todos os restaurante
      8 - Sair
      '''
  )
    opcao = input('O que gostaria de fazer? Escolha um número entre as opções')

    # Executa o comando para limpar a saída
    clear_output(wait = True)

    match opcao:
      case '1':
        cadastrar_restaurante(pasta, arquivo)
      case '2':
        registrar_hist_vendas(pasta, arquivo)
      case '3':
        verificar_rest_proximos(pasta, arquivo)
      case '4':
        mostrar_restaurantes(pasta, arquivo)
      case '5':
        mostrar_cardapio(pasta, arquivo)
      case '6':
        excluir_restaurante(pasta, arquivo)
      case '7':
        excluir_tudo(pasta, arquivo + '.json')
      case '8':
        print('Sistema preparando-se para encerrar...')
        sair = True
      case _:
        print('Opção inválida!')
    input('Pressione qualquer tecla para continuar...')
  # Fim do while
# Fim da função principal

Sistema preparando-se para encerrar...
Pressione qualquer tecla para continuar...
